In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Downloads/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
links = pd.read_csv('data/export.csv')
links

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86812,144122,https://forum.effectivealtruism.org/posts/A49f...,Potential Theories of Change for the Animal Ad...,\N,\N,2022-02-27 15:29:04.266921+00,2022-02-27 15:29:04.266935+00,NaN,\N,Custom,98.0,0.0,NaN
86813,144123,https://www.cnbc.com/2018/06/20/harvard-study-...,Harvard study: What CEOs do all day,\N,\N,2022-02-27 15:29:04.308017+00,2022-02-27 15:29:04.30803+00,NaN,\N,Custom,84.0,0.0,NaN
86814,144124,https://www.mckinsey.com/business-functions/st...,Mindsets and practices of the best CEOs,\N,\N,2022-02-27 15:29:04.351363+00,2022-02-27 15:29:04.351374+00,NaN,\N,Custom,81.0,0.0,NaN
86815,144125,https://www.inc.com/joseph-steinberg/7-ways-su...,7 Ways Successful CEOs Improve the Performance...,\N,\N,2022-02-27 15:29:04.390718+00,2022-02-27 15:29:04.39073+00,NaN,\N,Custom,43.0,0.0,NaN


In [2]:
links['added'].max()

'2022-02-27 17:29:47.077772+00'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
/Users/peterhurford/.virtualenvs/dev/lib/python3.8/site-packages/pandas/core/computation/expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
86812    144122
86813    144123
86814    144124
86815    144125
86816    144126
Name: id, Length: 7119, dtype: int64

In [6]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
84232,141501,"""https://www.centreforeffectivealtruism.org/bl...",CEA’s 2017 review and 2018 plans,\N,\N,2022-02-14 17:10:03.266418+00,2022-02-14 17:10:03.266418+00,NaN,\N,CEA,4.0,0.0,\N
84530,141833,"""https://www.centreforeffectivealtruism.org/bl...",CEA’s 2017 review and 2018 plans,\N,\N,2022-02-20 15:59:48.912259+00,2022-02-20 15:59:48.912259+00,NaN,\N,CEA,31.0,0.0,\N
84536,141839,http://www.rootsofprogress.org/event-doing-bet...,Event: Doing Better at Staying Optimistic with...,\N,\N,2022-02-23 04:52:06.208413+00,2022-02-23 04:52:06.208413+00,NaN,\N,Progress,17.0,0.0,\N
84537,141840,https://danwahl.net/blog/lightfoot-prospiracy,The Lightfoot Prospiracy,\N,\N,2022-02-23 04:52:06.211656+00,2022-02-23 04:52:06.211656+00,NaN,\N,DanWahl,34.0,0.0,\N
84538,141841,https://acesounderglass.com/2021/12/23/interne...,Internet Literacy Atrophy,\N,\N,2022-02-23 04:52:06.214236+00,2022-02-23 04:52:06.214236+00,NaN,\N,VanNostrand,28.0,0.0,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86772,144082,https://www.facebook.com/messages/t/1000012985...,Messenger,\N,\N,2022-02-27 15:29:02.613007+00,2022-02-27 15:29:02.613018+00,NaN,\N,Custom,89.0,0.0,NaN
86795,144105,https://docs.google.com/document/d/1IRdZlBJGzh...,Ways RP could facilitate AIGS work by non-RP p...,\N,docs.google.com,2022-02-27 15:29:03.566991+00,2022-02-27 21:31:26.726683+00,-1.0,\N,Custom,98.0,0.0,NaN
86802,144112,https://www.benkuhn.net/listen/,To listen well get curious,\N,\N,2022-02-27 15:29:03.864609+00,2022-02-27 15:29:03.864622+00,NaN,\N,Custom,52.0,0.0,NaN
86812,144122,https://forum.effectivealtruism.org/posts/A49f...,Potential Theories of Change for the Animal Ad...,\N,\N,2022-02-27 15:29:04.266921+00,2022-02-27 15:29:04.266935+00,NaN,\N,Custom,98.0,0.0,NaN


In [7]:
links[links['url'].duplicated()]['id']

84232    141501
84530    141833
84536    141839
84537    141840
84538    141841
          ...  
86772    144082
86795    144105
86802    144112
86812    144122
86814    144124
Name: id, Length: 1938, dtype: int64

In [14]:
link_file = open('/Users/peterhurford/Downloads/links.txt', 'r')
new_links = link_file.readlines()

In [15]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

...1/193
...11/193
...21/193
...31/193
...41/193
...51/193
...61/193
...71/193
...81/193
...91/193
...101/193
...111/193
...121/193
...131/193
...141/193
...151/193
...161/193
...171/193
...181/193
...191/193
...193/193


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17.0,0.0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59.0,0.0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56.0,0.0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29.0,0.0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86812,144122,https://forum.effectivealtruism.org/posts/A49f...,Potential Theories of Change for the Animal Ad...,\N,\N,2022-02-27 15:29:04.266921,2022-02-27 15:29:04.266935,\N,\N,Custom,98.0,0.0,NaN
86813,144123,https://www.cnbc.com/2018/06/20/harvard-study-...,Harvard study: What CEOs do all day,\N,\N,2022-02-27 15:29:04.308017,2022-02-27 15:29:04.308030,\N,\N,Custom,84.0,0.0,NaN
86814,144124,https://www.mckinsey.com/business-functions/st...,Mindsets and practices of the best CEOs,\N,\N,2022-02-27 15:29:04.351363,2022-02-27 15:29:04.351374,\N,\N,Custom,81.0,0.0,NaN
86815,144125,https://www.inc.com/joseph-steinberg/7-ways-su...,7 Ways Successful CEOs Improve the Performance...,\N,\N,2022-02-27 15:29:04.390718,2022-02-27 15:29:04.390730,\N,\N,Custom,43.0,0.0,NaN


In [16]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,10.0,0.0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,productivity,Custom,9.0,0.0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,effectivealtruism,Custom,85.0,0.0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1,politicalscience,Custom,82.0,0.0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0,entrepreneurship,Custom,59.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86812,144122,https://forum.effectivealtruism.org/posts/A49f...,Potential Theories of Change for the Animal Ad...,\N,\N,2022-02-27 15:29:04.266921,2022-02-27 15:29:04.266935,\N,\N,Custom,98.0,0.0,NaN
86813,144123,https://www.cnbc.com/2018/06/20/harvard-study-...,Harvard study: What CEOs do all day,\N,\N,2022-02-27 15:29:04.308017,2022-02-27 15:29:04.308030,\N,\N,Custom,84.0,0.0,NaN
86814,144124,https://www.mckinsey.com/business-functions/st...,Mindsets and practices of the best CEOs,\N,\N,2022-02-27 15:29:04.351363,2022-02-27 15:29:04.351374,\N,\N,Custom,81.0,0.0,NaN
86815,144125,https://www.inc.com/joseph-steinberg/7-ways-su...,7 Ways Successful CEOs Improve the Performance...,\N,\N,2022-02-27 15:29:04.390718,2022-02-27 15:29:04.390730,\N,\N,Custom,43.0,0.0,NaN


In [17]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86812,144122,https://forum.effectivealtruism.org/posts/A49f...,Potential Theories of Change for the Animal Ad...,\N,\N,2022-02-27 15:29:04.266921,2022-02-27 15:29:04.266935,\N,\N,Custom,98,0,NaN
86813,144123,https://www.cnbc.com/2018/06/20/harvard-study-...,Harvard study: What CEOs do all day,\N,\N,2022-02-27 15:29:04.308017,2022-02-27 15:29:04.308030,\N,\N,Custom,84,0,NaN
86814,144124,https://www.mckinsey.com/business-functions/st...,Mindsets and practices of the best CEOs,\N,\N,2022-02-27 15:29:04.351363,2022-02-27 15:29:04.351374,\N,\N,Custom,81,0,NaN
86815,144125,https://www.inc.com/joseph-steinberg/7-ways-su...,7 Ways Successful CEOs Improve the Performance...,\N,\N,2022-02-27 15:29:04.390718,2022-02-27 15:29:04.390730,\N,\N,Custom,43,0,NaN


In [18]:
links.to_csv('data/export.csv', index=False)